# Lab 05 — Linear Models

> **강의 시간:** 약 2시간  
> **주제:** 선형 회귀 이론부터 PyTorch 구현까지

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | 선형 회귀 수식과 Closed-form Solution 이해 | 25분 |
| 2 | MSE / MAE 손실 함수의 특성 비교 | 20분 |
| 3 | Batch / SGD / Mini-batch 경사 하강법 구현 및 비교 | 35분 |
| 4 | PyTorch `autograd`, `nn.Linear`로 집값 예측 모델 구현 | 30분 |
| 5 | 다항 특성, Ridge / Lasso 정규화 개념 | 10분 |

---

**데이터셋:** California Housing (sklearn) — 캘리포니아 지역별 주택 가격

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
torch.manual_seed(42)
print('PyTorch:', torch.__version__)

---
## Part 1. 선형 회귀 이론

### 1-1. 문제 정의

**회귀(Regression)** 란 연속적인 수치 출력을 예측하는 지도 학습 문제입니다.

선형 회귀는 입력 특성 $\mathbf{x}$와 출력 $y$ 사이를 **선형 함수**로 모델링합니다.

$$\hat{y} = w_1 x_1 + w_2 x_2 + \cdots + w_d x_d + b = \mathbf{w}^\top \mathbf{x} + b$$

- $\mathbf{w}$ : 가중치(weight) — 각 특성이 예측에 미치는 영향
- $b$ : 편향(bias) — 절편

전체 데이터셋 행렬 표기:

$$\hat{\mathbf{y}} = X\mathbf{w} + b, \quad X \in \mathbb{R}^{n \times d}$$

In [ ]:
# 1D 예제: 방 크기 → 집값
# 실제 관계: price = 3.0 * size + 5.0 + noise
n = 80
size  = rng.uniform(20, 100, n)
price = 3.0 * size + 5.0 + rng.normal(0, 8, n)

plt.figure(figsize=(7, 4))
plt.scatter(size, price, alpha=0.7, edgecolors='k', s=50)
plt.xlabel('방 크기 (m²)')
plt.ylabel('집값 (만 달러)')
plt.title('1D 선형 회귀 예제 — 방 크기 vs 집값')
plt.tight_layout()
plt.show()
print(f'샘플 수: {n}')
print('실제 관계: price = 3.0 * size + 5.0 + noise')

### 1-2. Closed-form Solution (정규 방정식)

MSE 손실을 최소화하는 $\mathbf{w}$를 **해석적으로** 구할 수 있습니다.

$$\mathcal{L}(\mathbf{w}) = \frac{1}{n} \|X\mathbf{w} - \mathbf{y}\|^2$$

$\nabla_\mathbf{w} \mathcal{L} = 0$ 으로 놓으면:

$$\boxed{\mathbf{w}^* = (X^\top X)^{-1} X^\top \mathbf{y}}$$

> **언제 쓸 수 있나?** 특성 수 $d$가 작을 때 ($d < 10{,}000$).  
> 역행렬 계산이 $\mathcal{O}(d^3)$이므로 고차원에서는 비실용적.

In [ ]:
# 정규 방정식 직접 구현
# 편향 항을 위해 X에 1열 추가: X_b = [1 | x]
X_b = np.column_stack([np.ones(n), size])   # (80, 2)
y   = price

# w* = (X^T X)^{-1} X^T y
w_star = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
b_hat, w_hat = w_star[0], w_star[1]

print(f'추정 기울기 w : {w_hat:.4f}  (실제: 3.0)')
print(f'추정 절편   b : {b_hat:.4f}  (실제: 5.0)')

# 예측 및 시각화
x_line = np.linspace(20, 100, 200)
y_line = w_hat * x_line + b_hat
y_pred_ne = w_hat * size + b_hat
mse_ne    = np.mean((y_pred_ne - price)**2)
r2_ne     = 1 - np.sum((price - y_pred_ne)**2) / np.sum((price - price.mean())**2)

plt.figure(figsize=(7, 4))
plt.scatter(size, price, alpha=0.6, edgecolors='k', s=40, label='데이터')
plt.plot(x_line, y_line, color='tomato', lw=2.5,
         label=f'정규 방정식: y = {w_hat:.2f}x + {b_hat:.2f}')
for xi, yi, ypi in zip(size, price, y_pred_ne):
    plt.plot([xi, xi], [yi, ypi], color='gray', alpha=0.3, lw=0.8)
plt.xlabel('방 크기 (m²)')
plt.ylabel('집값 (만 달러)')
plt.title(f'Closed-form Solution  MSE={mse_ne:.2f}  R²={r2_ne:.4f}')
plt.legend()
plt.tight_layout()
plt.show()

### 1-3. 다중 선형 회귀 (Multiple Linear Regression)

실제 집값에는 여러 특성이 영향을 미칩니다.  
California Housing 데이터셋을 불러와 다중 회귀를 수행합니다.

In [ ]:
housing = fetch_california_housing(as_frame=True)
df = housing.frame.copy()

print('특성 수  :', len(housing.feature_names))
print('샘플 수  :', len(df))
print('\n특성 설명:')
descriptions = {
    'MedInc'    : '중위 소득 (만 달러)',
    'HouseAge'  : '주택 연령 (년)',
    'AveRooms'  : '평균 방 수',
    'AveBedrms' : '평균 침실 수',
    'Population': '인구 수',
    'AveOccup'  : '평균 거주자 수',
    'Latitude'  : '위도',
    'Longitude' : '경도',
}
for feat, desc in descriptions.items():
    print(f'  {feat:<12}: {desc}')
print('\n목표값: MedHouseVal — 중위 주택 가격 (×$100,000)')
df.head()

In [ ]:
# 분포 및 상관관계
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].hist(df['MedHouseVal'], bins=50, color='steelblue', edgecolor='white')
axes[0].axvline(df['MedHouseVal'].median(), color='tomato', lw=2,
                label=f"중앙값 {df['MedHouseVal'].median():.2f}")
axes[0].set_title('주택 가격 분포')
axes[0].set_xlabel('중위 주택 가격 (×$100K)')
axes[0].legend()

corr = df.corr().round(2)
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            vmin=-1, vmax=1, ax=axes[1], linewidths=0.3, annot_kws={'size': 7})
axes[1].set_title('특성 간 상관관계')
plt.tight_layout()
plt.show()

In [ ]:
# 정규 방정식으로 다중 선형 회귀
X_multi = df[housing.feature_names].values
y_multi = df['MedHouseVal'].values
X_multi_b = np.column_stack([np.ones(len(X_multi)), X_multi])

# lstsq: 수치적으로 더 안정적인 최소제곱법
w_multi = np.linalg.lstsq(X_multi_b, y_multi, rcond=None)[0]

print('다중 회귀 계수:')
print(f'  편향(b)   : {w_multi[0]:.4f}')
for feat, w in zip(housing.feature_names, w_multi[1:]):
    print(f'  {feat:<12}: {w:+.4f}')

y_pred_multi = X_multi_b @ w_multi
print(f'\nTrain MSE : {mean_squared_error(y_multi, y_pred_multi):.4f}')
print(f'Train R²  : {r2_score(y_multi, y_pred_multi):.4f}')

---
## Part 2. 손실 함수 (Loss Functions)

### 2-1. MSE vs MAE vs Huber

| 손실 함수 | 수식 | 특징 |
|---|---|---|
| **MSE** | $\frac{1}{n}\sum(y_i - \hat{y}_i)^2$ | 이상치에 민감, 미분 가능 |
| **MAE** | $\frac{1}{n}\sum|y_i - \hat{y}_i|$ | 이상치에 강건 |
| **Huber** | MSE + MAE 결합 | 두 장점 모두 |
| **RMSE** | $\sqrt{\text{MSE}}$ | 단위가 $y$와 동일 |
| **R²** | $1 - \frac{\text{SS}_{res}}{\text{SS}_{tot}}$ | 1에 가까울수록 좋음 |

In [ ]:
# 이상치 존재 시 MSE vs MAE 민감도 비교
np.random.seed(0)
x_toy = np.linspace(0, 10, 30)
y_toy = 2*x_toy + 1 + np.random.randn(30)*2
x_out = np.append(x_toy, [2, 4])
y_out = np.append(y_toy, [30, -10])   # 극단 이상치

def fit_line(x, y):
    Xb = np.column_stack([np.ones_like(x), x])
    return np.linalg.lstsq(Xb, y, rcond=None)[0]

w_clean   = fit_line(x_toy, y_toy)
x_line2   = np.linspace(0, 10, 200)

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for ax, (x_d, y_d, title) in zip(axes, [
    (x_toy, y_toy, '이상치 없음'),
    (x_out, y_out, '이상치 포함'),
]):
    ax.scatter(x_d[:30], y_d[:30], color='steelblue', s=40, zorder=3)
    if len(x_d) > 30:
        ax.scatter(x_d[30:], y_d[30:], color='red', s=80, marker='*', zorder=4, label='이상치')
    w = fit_line(x_d, y_d)
    ax.plot(x_line2, w[1]*x_line2+w[0], 'tomato', lw=2, label=f'MSE 최적선 (w={w[1]:.2f})')
    ax.plot(x_line2, w_clean[1]*x_line2+w_clean[0], 'green', lw=1.5, linestyle='--',
            label=f'클린 선 (w={w_clean[1]:.2f})')
    ax.set_title(title)
    ax.legend(fontsize=8)
plt.suptitle('MSE는 이상치에 민감하다', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 손실 함수 시각화
err   = np.linspace(-4, 4, 300)
delta = 1.0
mse_v   = err**2
mae_v   = np.abs(err)
huber_v = np.where(np.abs(err) <= delta, 0.5*err**2, delta*(np.abs(err)-0.5*delta))

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(err, mse_v,   label='MSE',            color='steelblue', lw=2)
axes[0].plot(err, mae_v,   label='MAE',            color='tomato',    lw=2)
axes[0].plot(err, huber_v, label=f'Huber (d={delta})', color='seagreen', lw=2)
axes[0].set_title('손실 함수 비교')
axes[0].set_xlabel('오차 (y - ŷ)')
axes[0].set_ylabel('손실')
axes[0].set_ylim(-0.5, 8)
axes[0].legend()

grad_mse   = 2*err
grad_mae   = np.sign(err)
grad_huber = np.where(np.abs(err) <= delta, err, delta*np.sign(err))
axes[1].plot(err, grad_mse,   label='dMSE/de',   color='steelblue', lw=2)
axes[1].plot(err, grad_mae,   label='dMAE/de',   color='tomato',    lw=2)
axes[1].plot(err, grad_huber, label='dHuber/de', color='seagreen',  lw=2)
axes[1].set_title('그래디언트 크기 비교')
axes[1].set_xlabel('오차 (y - ŷ)')
axes[1].set_ylabel('그래디언트')
axes[1].set_ylim(-3, 3)
axes[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# PyTorch 손실 함수 사용
y_true_t = torch.tensor([3.0, 5.0, 2.5, 7.0])
y_pred_t = torch.tensor([2.8, 5.5, 2.0, 6.5])

print(f'MSE   Loss : {nn.MSELoss()(y_pred_t, y_true_t).item():.4f}')
print(f'MAE   Loss : {nn.L1Loss()(y_pred_t, y_true_t).item():.4f}')
print(f'Huber Loss : {nn.HuberLoss(delta=1.0)(y_pred_t, y_true_t).item():.4f}')

def regression_metrics(y_true, y_pred, label=''):
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f'{label}')
    print(f'  MSE  : {mse:.4f}')
    print(f'  RMSE : {rmse:.4f}')
    print(f'  MAE  : {mae:.4f}')
    print(f'  R²   : {r2:.4f}')
    return dict(mse=mse, rmse=rmse, mae=mae, r2=r2)

regression_metrics(y_multi, y_pred_multi, '다중 선형 회귀 (정규 방정식)')

---
## Part 3. 경사 하강법 (Gradient Descent)

### 3-1. 핵심 아이디어

정규 방정식은 고차원에서 역행렬 계산이 비실용적입니다.  
**경사 하강법**은 손실을 반복적으로 줄이며 최적해를 찾습니다.

$$\mathbf{w} \leftarrow \mathbf{w} - \eta \cdot \nabla_\mathbf{w} \mathcal{L}$$

MSE의 그래디언트:

$$\nabla_\mathbf{w} \mathcal{L}_{\text{MSE}} = \frac{2}{n} X^\top (X\mathbf{w} - \mathbf{y})$$

| 방법 | 배치 크기 | 장점 | 단점 |
|---|---|---|---|
| **Batch GD** | 전체 $n$ | 안정적 수렴 | 느림, 메모리 큼 |
| **SGD** | 1개 | 빠름, 지역 최소 탈출 | 불안정, 진동 큼 |
| **Mini-batch GD** | $k$개 | 두 장점 균형 | 배치 크기 튜닝 필요 |

In [ ]:
# 1D 예제로 3가지 경사 하강법 구현
X_gd    = size.reshape(-1, 1)
y_gd    = price
x_mean, x_std = X_gd.mean(), X_gd.std()
y_mean, y_std = y_gd.mean(),  y_gd.std()
X_norm  = (X_gd - x_mean) / x_std
y_norm  = (y_gd - y_mean) / y_std
n_gd    = len(y_norm)

def gd_step(w, b, X, y, lr):
    n_   = len(y)
    err  = X.ravel()*w + b - y
    dw   = (2/n_) * (err * X.ravel()).sum()
    db   = (2/n_) * err.sum()
    return w - lr*dw, b - lr*db

def mse_val(w, b, X, y):
    return np.mean((X.ravel()*w + b - y)**2)

EPOCHS, LR, BATCH = 200, 0.05, 16

# Batch GD
w_b, b_b = 0.0, 0.0
loss_batch = []
for _ in range(EPOCHS):
    w_b, b_b = gd_step(w_b, b_b, X_norm, y_norm, LR)
    loss_batch.append(mse_val(w_b, b_b, X_norm, y_norm))

# SGD
w_s, b_s = 0.0, 0.0
loss_sgd  = []
for ep in range(EPOCHS):
    idx = rng.permutation(n_gd)
    for i in idx:
        w_s, b_s = gd_step(w_s, b_s, X_norm[i:i+1], y_norm[i:i+1], LR)
    loss_sgd.append(mse_val(w_s, b_s, X_norm, y_norm))

# Mini-batch GD
w_m, b_m = 0.0, 0.0
loss_mini = []
for ep in range(EPOCHS):
    idx = rng.permutation(n_gd)
    for start in range(0, n_gd, BATCH):
        bi = idx[start:start+BATCH]
        w_m, b_m = gd_step(w_m, b_m, X_norm[bi], y_norm[bi], LR)
    loss_mini.append(mse_val(w_m, b_m, X_norm, y_norm))

print(f'최종 손실 - Batch:{loss_batch[-1]:.5f}  SGD:{loss_sgd[-1]:.5f}  Mini:{loss_mini[-1]:.5f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(loss_batch, label='Batch GD',   color='steelblue', lw=2)
axes[0].plot(loss_mini,  label='Mini-batch', color='seagreen',  lw=2)
axes[0].plot(loss_sgd,   label='SGD',        color='tomato',    lw=1.5, alpha=0.8)
axes[0].set_title('3가지 경사 하강법 손실 수렴 비교')
axes[0].set_xlabel('에폭')
axes[0].set_ylabel('MSE Loss')
axes[0].set_yscale('log')
axes[0].legend()

axes[1].plot(loss_batch[:30], label='Batch GD',   color='steelblue', lw=2)
axes[1].plot(loss_mini[:30],  label='Mini-batch', color='seagreen',  lw=2)
axes[1].plot(loss_sgd[:30],   label='SGD',        color='tomato',    lw=1.5, alpha=0.8)
axes[1].set_title('초기 수렴 속도 (첫 30 에폭)')
axes[1].set_xlabel('에폭')
axes[1].set_ylabel('MSE Loss')
axes[1].legend()
plt.tight_layout()
plt.show()

### 3-2. 학습률(Learning Rate) 영향

학습률 $\eta$는 가장 중요한 하이퍼파라미터입니다.

- $\eta$ **너무 큼** → 발산(divergence)
- $\eta$ **너무 작음** → 느린 수렴
- $\eta$ **적절** → 안정적, 빠른 수렴

In [ ]:
learning_rates = [0.001, 0.05, 0.2, 0.8]
colors_lr      = ['purple', 'steelblue', 'seagreen', 'tomato']
EPOCHS_LR      = 100

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

for lr, color in zip(learning_rates, colors_lr):
    w, b = 0.0, 0.0
    losses_lr = []
    for _ in range(EPOCHS_LR):
        w, b = gd_step(w, b, X_norm, y_norm, lr)
        loss = mse_val(w, b, X_norm, y_norm)
        if loss > 1e6 or np.isnan(loss):
            losses_lr.extend([np.nan] * (EPOCHS_LR - len(losses_lr)))
            break
        losses_lr.append(loss)
    axes[0].plot(losses_lr, label=f'lr={lr}', color=color, lw=2)

axes[0].set_title('학습률에 따른 수렴 특성')
axes[0].set_xlabel('에폭')
axes[0].set_ylabel('MSE Loss')
axes[0].set_ylim(-0.1, 3)
axes[0].legend()

# 손실 등고선 + 경사 하강 경로
W_r = np.linspace(-2, 2, 80)
B_r = np.linspace(-2, 2, 80)
WW, BB = np.meshgrid(W_r, B_r)
LL = np.array([[mse_val(wi, bi, X_norm, y_norm) for wi, bi in zip(Wr, Br)]
               for Wr, Br in zip(WW, BB)])
axes[1].contourf(WW, BB, LL, levels=25, cmap='RdYlGn_r', alpha=0.8)
for lr, color in zip([0.05, 0.2, 0.8], ['steelblue', 'seagreen', 'tomato']):
    w, b = -1.5, -1.5
    wh, bh = [w], [b]
    for _ in range(20):
        w, b = gd_step(w, b, X_norm, y_norm, lr)
        if abs(w) > 5 or abs(b) > 5: break
        wh.append(w); bh.append(b)
    axes[1].plot(wh, bh, 'o-', color=color, markersize=4, lw=1.5, label=f'lr={lr}')
axes[1].set_title('손실 등고선 + 경사 하강 경로 (20스텝)')
axes[1].set_xlabel('w'); axes[1].set_ylabel('b')
axes[1].legend(fontsize=8)
plt.tight_layout()
plt.show()

---
## Part 4. PyTorch로 선형 회귀 구현

### 4-1. PyTorch `autograd` 기초

PyTorch는 **자동 미분(autograd)** 을 제공합니다.  
`requires_grad=True`로 설정된 텐서는 연산 그래프를 기록하고,  
`.backward()` 호출 시 연쇄 법칙으로 그래디언트를 자동 계산합니다.

```
순전파: x → y_pred → loss
역전파: loss.backward() → w.grad, b.grad 자동 계산
업데이트: w = w - lr * w.grad
```

In [ ]:
# autograd 동작 원리
x_ag  = torch.tensor([2.0, 3.0, 4.0])
w_ag  = torch.tensor(1.5, requires_grad=True)
b_ag  = torch.tensor(0.0, requires_grad=True)
y_tgt = torch.tensor([5.0, 7.0, 9.0])   # w=2, b=1이면 완벽

# 순전파
y_hat_ag = w_ag * x_ag + b_ag
loss_ag  = ((y_hat_ag - y_tgt)**2).mean()
print(f'예측: {y_hat_ag.tolist()}')
print(f'손실: {loss_ag.item():.4f}')

# 역전파
loss_ag.backward()
print(f'\ndL/dw = {w_ag.grad.item():.4f}  (이 방향으로 w를 줄이면 손실 감소)')
print(f'dL/db = {b_ag.grad.item():.4f}')

# 수동 검증
e = y_hat_ag.detach().numpy() - y_tgt.numpy()
dw_manual = 2/3 * (e * x_ag.numpy()).sum()
print(f'\n수동 계산 dL/dw = {dw_manual:.4f}  (일치 확인)')

### 4-2. 데이터 준비

In [ ]:
X_all = df[housing.feature_names].values.astype(np.float32)
y_all = df['MedHouseVal'].values.astype(np.float32)

X_tv, X_te, y_tv, y_te = train_test_split(X_all, y_all, test_size=0.2, random_state=42)
X_tr, X_va, y_tr, y_va = train_test_split(X_tv,  y_tv,  test_size=0.25, random_state=42)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_tr_s = scaler_X.fit_transform(X_tr)
X_va_s = scaler_X.transform(X_va)
X_te_s = scaler_X.transform(X_te)
y_tr_s = scaler_y.fit_transform(y_tr.reshape(-1,1)).ravel()
y_va_s = scaler_y.transform(y_va.reshape(-1,1)).ravel()
y_te_s = scaler_y.transform(y_te.reshape(-1,1)).ravel()

def make_loader(X, y, batch_size=256, shuffle=False):
    ds = TensorDataset(torch.tensor(X), torch.tensor(y))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = make_loader(X_tr_s, y_tr_s, 256, shuffle=True)
val_loader   = make_loader(X_va_s, y_va_s, 256)
test_loader  = make_loader(X_te_s, y_te_s, 256)

print(f'Train: {len(X_tr_s):>6}개  ({len(X_tr_s)/len(X_all)*100:.0f}%)')
print(f'Val  : {len(X_va_s):>6}개  ({len(X_va_s)/len(X_all)*100:.0f}%)')
print(f'Test : {len(X_te_s):>6}개  ({len(X_te_s)/len(X_all)*100:.0f}%)')

### 4-3. `nn.Linear` 모델 정의

`nn.Linear(in, out)`은 내부적으로 `y = xW^T + b` 를 수행합니다.

```python
# 직접 구현하면 이렇게 됩니다:
y = x @ W.T + b   # W: (out, in),  b: (out,)
```

In [ ]:
class LinearRegression(nn.Module):
    """단순 선형 회귀: y = Xw + b"""
    def __init__(self, in_features: int):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)   # 출력 1개 (회귀)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x).squeeze(1)          # (batch,1) → (batch,)

model = LinearRegression(in_features=8)
print(model)
print(f'\n파라미터 수: {sum(p.numel() for p in model.parameters())}')
print(f'  가중치 w : {model.linear.weight.shape}')
print(f'  편향   b : {model.linear.bias.shape}')

x_sample = torch.tensor(X_tr_s[:4])
with torch.no_grad():
    y_sample = model(x_sample)
print(f'\n초기 예측 (랜덤 가중치): {y_sample.numpy().round(3)}')

### 4-4. 학습 루프 (Train / Validate)

매 에폭마다 아래 5단계를 반복합니다:

```
1. optimizer.zero_grad()   ← 이전 그래디언트 초기화
2. y_hat = model(x)        ← 순전파
3. loss  = criterion(...)  ← 손실 계산
4. loss.backward()         ← 역전파
5. optimizer.step()        ← 파라미터 업데이트
```

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()          # 1
        y_hat = model(xb)              # 2
        loss  = criterion(y_hat, yb)   # 3
        loss.backward()                # 4
        optimizer.step()               # 5
        total_loss += loss.item() * len(xb)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total = 0
    for xb, yb in loader:
        total += criterion(model(xb), yb).item() * len(xb)
    return total / len(loader.dataset)

model_pt  = LinearRegression(8)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_pt.parameters(), lr=1e-2)

EPOCHS = 100
train_losses_pt, val_losses_pt = [], []
for epoch in range(1, EPOCHS+1):
    tr = train_one_epoch(model_pt, train_loader, criterion, optimizer)
    va = evaluate(model_pt, val_loader, criterion)
    train_losses_pt.append(tr)
    val_losses_pt.append(va)
    if epoch % 20 == 0:
        print(f'Epoch {epoch:3d} | Train MSE: {tr:.4f} | Val MSE: {va:.4f}')
print('\n학습 완료')

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(train_losses_pt, label='Train', color='steelblue')
axes[0].plot(val_losses_pt,   label='Val',   color='tomato')
axes[0].set_title('학습 / 검증 손실 곡선')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE (표준화 공간)')
axes[0].legend()

model_pt.eval()
with torch.no_grad():
    y_pred_s = model_pt(torch.tensor(X_te_s)).numpy()
y_pred_orig = scaler_y.inverse_transform(y_pred_s.reshape(-1,1)).ravel()

axes[1].scatter(y_te, y_pred_orig, alpha=0.3, s=10, color='steelblue')
lo = min(y_te.min(), y_pred_orig.min())
hi = max(y_te.max(), y_pred_orig.max())
axes[1].plot([lo,hi],[lo,hi],'r--',lw=2,label='완벽한 예측선')
axes[1].set_title('예측값 vs 실제값')
axes[1].set_xlabel('실제 집값 (×$100K)')
axes[1].set_ylabel('예측 집값 (×$100K)')
axes[1].legend()

residuals = y_te - y_pred_orig
axes[2].hist(residuals, bins=50, color='steelblue', edgecolor='white')
axes[2].axvline(0, color='red', lw=2)
axes[2].axvline(residuals.mean(), color='orange', lw=2,
                label=f'평균={residuals.mean():.3f}')
axes[2].set_title('잔차 분포')
axes[2].set_xlabel('잔차 (실제 - 예측)')
axes[2].legend(fontsize=8)
plt.tight_layout()
plt.show()

regression_metrics(y_te, y_pred_orig, '\n[ Test 성능 — PyTorch 선형 회귀 ]')

In [ ]:
# 학습된 가중치 해석
w_learned = model_pt.linear.weight.detach().numpy().ravel()
b_learned = model_pt.linear.bias.detach().item()

plt.figure(figsize=(8, 4))
colors_w = ['tomato' if w > 0 else 'steelblue' for w in w_learned]
plt.barh(housing.feature_names, w_learned, color=colors_w)
plt.axvline(0, color='black', lw=0.8)
plt.title('학습된 선형 회귀 가중치\n(빨강=양의 영향, 파랑=음의 영향)')
plt.xlabel('가중치 크기 (표준화 공간)')
plt.tight_layout()
plt.show()

print('가중치 해석:')
for feat, w in sorted(zip(housing.feature_names, w_learned), key=lambda x: abs(x[1]), reverse=True):
    sign = '(+) 집값 상승' if w > 0 else '(-) 집값 하락'
    print(f'  {feat:<12}: {w:+.4f}  {sign}')

### 4-5. 옵티마이저 비교 (SGD vs Adam vs RMSprop)

In [ ]:
configs = [
    ('SGD (lr=0.05)',     lambda p: torch.optim.SGD(p,     lr=0.05)),
    ('Adam (lr=0.01)',    lambda p: torch.optim.Adam(p,    lr=0.01)),
    ('RMSprop (lr=0.01)', lambda p: torch.optim.RMSprop(p, lr=0.01)),
]
colors_opt = ['steelblue', 'tomato', 'seagreen']

plt.figure(figsize=(8, 4))
for (name, opt_fn), color in zip(configs, colors_opt):
    m   = LinearRegression(8)
    opt = opt_fn(m.parameters())
    vh  = [evaluate(m, val_loader, criterion)]
    for _ in range(60):
        train_one_epoch(m, train_loader, criterion, opt)
        vh.append(evaluate(m, val_loader, criterion))
    plt.plot(vh, label=name, color=color, lw=2)
plt.title('옵티마이저별 검증 손실 수렴 비교')
plt.xlabel('Epoch')
plt.ylabel('Val MSE')
plt.legend()
plt.tight_layout()
plt.show()

---
## Part 5. 모델 개선

### 5-1. 다항 특성 (Polynomial Features)

선형 모델이지만 **입력 특성을 고차항으로 확장**하면 비선형 관계를 포착할 수 있습니다.

$$x \rightarrow [1,\; x,\; x^2,\; x^3, \ldots, x^d]$$

단, 차수가 높아질수록 **과적합(overfitting)** 위험이 증가합니다.

In [ ]:
np.random.seed(1)
x_poly = np.linspace(-3, 3, 80)
y_poly = 0.5*x_poly**3 - 2*x_poly**2 + x_poly + 3 + np.random.randn(80)*2
x_eval = np.linspace(-3, 3, 300)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, deg in zip(axes, [1, 3, 9]):
    Xp = np.column_stack([x_poly**d for d in range(deg+1)])
    wp = np.linalg.lstsq(Xp, y_poly, rcond=None)[0]
    Xe = np.column_stack([x_eval**d for d in range(deg+1)])
    ye = Xe @ wp
    r2 = r2_score(y_poly, Xp@wp)
    ax.scatter(x_poly, y_poly, alpha=0.5, s=20, color='steelblue')
    ax.plot(x_eval, ye, color='tomato', lw=2)
    ax.set_ylim(-25, 20)
    ax.set_title(f'{deg}차 다항식  R²={r2:.3f}')
    label = '과소적합' if deg==1 else ('적합' if deg==3 else '과적합')
    ax.text(0, -22, label, ha='center', fontsize=12,
            color='red' if label!='적합' else 'green', fontweight='bold')
plt.suptitle('다항 특성 — 과소적합 vs 적합 vs 과적합', y=1.02)
plt.tight_layout()
plt.show()

### 5-2. 정규화 (Regularization) — Ridge & Lasso

과적합을 방지하기 위해 손실 함수에 **페널티 항**을 추가합니다.

$$\text{Ridge (L2)}: \quad \mathcal{L} + \lambda \|\mathbf{w}\|_2^2$$

$$\text{Lasso (L1)}: \quad \mathcal{L} + \lambda \|\mathbf{w}\|_1$$

| | Ridge | Lasso |
|---|---|---|
| 패널티 | $\lambda\|w\|_2^2$ | $\lambda\|w\|_1$ |
| 효과 | 가중치 축소 | 가중치를 0으로 (특성 선택) |
| PyTorch | `weight_decay` 파라미터 | 수동 구현 필요 |

In [ ]:
# 고차 다항식 + Ridge 정규화
DEG = 9
Xhi = np.column_stack([x_poly**d for d in range(DEG+1)])
Xhi = (Xhi - Xhi.mean(0)) / (Xhi.std(0) + 1e-8)
Xev = np.column_stack([x_eval**d for d in range(DEG+1)])
Xev = (Xev - Xev.mean(0)) / (Xev.std(0) + 1e-8)

lambdas    = [0.0, 0.01, 1.0, 10.0]
colors_reg = plt.cm.Blues(np.linspace(0.4, 1.0, 4))

plt.figure(figsize=(10, 4))
plt.scatter(x_poly, y_poly, alpha=0.5, s=20, color='gray', zorder=3)
plt.plot(x_eval, 0.5*x_eval**3-2*x_eval**2+x_eval+3, 'k--', lw=1.5,
         label='실제 함수', zorder=4)
for lam, color in zip(lambdas, colors_reg):
    I = np.eye(Xhi.shape[1])
    wr = np.linalg.solve(Xhi.T@Xhi + lam*I, Xhi.T@y_poly)
    plt.plot(x_eval, Xev@wr, color=color, lw=2, label=f'Ridge l={lam}')
plt.ylim(-25, 20)
plt.title('Ridge 정규화 — λ 값에 따른 과적합 억제')
plt.xlabel('x'); plt.ylabel('y')
plt.legend(fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# PyTorch에서 Ridge 정규화 = weight_decay
print('[ 정규화 효과 비교 ]')
print(f'{"방법":<16}  {"Val MSE":>9}  {"Test MSE":>9}  {"||w||2":>8}')
print('-' * 50)
for name, wd in [('No Reg', 0.0), ('Ridge 1e-3', 1e-3), ('Ridge 1e-1', 1e-1)]:
    m   = LinearRegression(8)
    opt = torch.optim.Adam(m.parameters(), lr=0.01, weight_decay=wd)
    for _ in range(80):
        train_one_epoch(m, train_loader, criterion, opt)
    va = evaluate(m, val_loader, criterion)
    te = evaluate(m, test_loader, criterion)
    wn = m.linear.weight.detach().norm().item()
    print(f'{name:<16}  {va:>9.4f}  {te:>9.4f}  {wn:>8.4f}')

---
## Exercise

### Exercise 1. 정규 방정식 직접 구현

California Housing 데이터의 **처음 2개 특성** (`MedInc`, `HouseAge`)만 사용해  
정규 방정식 $\mathbf{w}^* = (X^\top X)^{-1} X^\top \mathbf{y}$를 직접 구현하고  
MSE와 R²를 출력하세요. (`np.linalg.lstsq` 미사용)

In [ ]:
# Your code here


### Exercise 2. Mini-batch 배치 크기 실험

**batch_size = 1, 16, 64, 전체(n)** 4가지 경우의 수렴 곡선을 하나의 그래프에 그리고  
최종 손실을 비교하세요.  
데이터는 위에서 사용한 1D 예제 (`X_norm`, `y_norm`)를 사용하세요.

In [ ]:
# Your code here


### Exercise 3. PyTorch 파이프라인 완성

아래 클래스와 학습 루프를 완성하세요.

- 모델: `nn.Linear(8, 1)` + **He 초기화** (`nn.init.kaiming_uniform_`) 적용
- 옵티마이저: `Adam`, `lr=0.01`
- 스케줄러: `StepLR(optimizer, step_size=30, gamma=0.5)` — 30 에폭마다 lr 절반
- 100 에폭 학습 후 **Test MSE와 R²** 출력
- 학습 곡선 (train loss, val loss)과 **학습률 변화** 함께 시각화

In [ ]:
class LinearRegressionV2(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)
        # Your code here: He 초기화 적용

    def forward(self, x):
        # Your code here
        pass


# Your code here: 학습 루프 + 스케줄러 + 시각화


---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **선형 회귀** | $\hat{y} = \mathbf{w}^\top\mathbf{x} + b$, 연속값 예측 |
| **정규 방정식** | $(X^\top X)^{-1}X^\top\mathbf{y}$, 소규모 데이터에 적합 |
| **MSE vs MAE** | MSE는 이상치에 민감, MAE는 강건 |
| **Batch GD** | 안정적이지만 느림, 전체 데이터 사용 |
| **SGD** | 빠르지만 진동, 1개 샘플 사용 |
| **Mini-batch** | 실제 딥러닝의 표준 방식 |
| **학습률 η** | 가장 중요한 하이퍼파라미터 |
| **autograd** | `requires_grad=True` → `.backward()` → `.grad` |
| **nn.Linear** | `Linear(in, out)` 내부: `xW^T + b` |
| **Ridge** | L2 정규화, `weight_decay`로 적용 |
| **Lasso** | L1 정규화, 특성 선택 효과 |

---

**다음 강의 (Week 6):** 분류 — Logistic Regression, 결정 경계, Precision/Recall, ROC-AUC